In [1]:
import paddle
import paddlehub as hub

print('paddle version : ', paddle.__version__)
print('paddlehub version : ', hub.__version__)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


paddle version :  2.2.0
paddlehub version :  2.0.4


In [2]:
from paddle.fluid.contrib.model_stat import summary
##模型加载
model = hub.Module(name="ernie", task='seq-cls', num_classes=14) 
print(model.summary)

[2021-11-16 14:15:00,238] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
W1116 14:15:00.242748  2875 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1116 14:15:00.248580  2875 device_context.cc:465] device: 0, cuDNN Version: 7.6.


Baidu's ERNIE, Enhanced Representation through kNowledge IntEgration, max_seq_len=512 when predtrained. The module is executed as paddle.dygraph.


In [3]:
# 数据集解压缩，并打印验证数据集前3条、后3条数据
%cd /home/aistudio/data/data16287/
!tar -zxvf thu_news.tar.gz
!ls -hl thu_news
!echo 'print the first 3 samples in the validation datasets'
!head -n 3 thu_news/valid.txt
!echo 'print the last 3 samples in the validation datasets'
!tail -n 3 thu_news/valid.txt

/home/aistudio/data/data16287
thu_news/
thu_news/test.txt
thu_news/valid.txt
thu_news/train.txt
总用量 30M
-rw-r--r-- 1 aistudio root 3.7M 11月 19  2019 test.txt
-rw-r--r-- 1 aistudio root  23M 11月 19  2019 train.txt
-rw-r--r-- 1 aistudio root 3.6M 11月 19  2019 valid.txt
print the first 3 samples in the validation datasets
text_a	label
直击特里勾手助小牛反超神鸟发威火箭仍处劣势　　新浪体育讯　北京时间4月16日消息，火箭今天迎来常规赛的收官战。客场挑战达拉斯小牛的比赛将关系到火箭队最终的排名，目前西部的竞争仍然非常激励。尤其是西部第二到第七的六支球队将在今天展开捉对厮杀的情况下，黄蜂、小牛、开拓者以及马刺都有可能是火箭队的下一个对手。以下为本场比赛的精彩瞬间——　　第四节比赛还剩下8分多钟，姚明重新回到场上，但是小牛队的霍里随后在右翼接队友传球后上演一记三分跳投，虽然身体动作已经变形，但是他仍然将球命中，随后基德在弧顶处的一记三分跳投帮助小牛终于将比分反超，火箭在进攻中直接打不开局面，而小牛更是利用特里空切后的一记勾手将比分反超至4分，包括库班在内的所有小牛现场球迷都站起来振臂高呼，而此时阿德尔曼还在场边低头深沉的思索中。　　比赛还剩下最后5分多钟，洛瑞带球突破中被小牛队员犯规，洛瑞来不及刹车让自己直接撞到了观众席上，右腿疼痛难忍的他脸都几乎变形，结果他两罚两中，火箭队还落后两分。随后，洛瑞将球直接传给此时已经回到场上的阿泰，野兽此时将球稳下，并没有急于进攻，随后他顺势将球塞给兰德里，神鸟利用对方站位空隙直接杀到篮下，上篮成功，火箭将比分终于追至80平。场上局面对于火箭来说绝对是不容有失。　　(sabrina)	体育
北京网购年消费额112亿元　　商报讯(记者吴文治)昨日，淘宝网发布的《2009-2010年度中国网购热门城市报告》显示，北京年度网购消费力达112.5亿元，与上海相差近62亿元，位列十大热门消费力城市第二位。此外，男性网购的消费

In [4]:
## 数据集加载
import os, io, csv
from paddlehub.datasets.base_nlp_dataset import InputExample, TextClassificationDataset

DATA_DIR="/home/aistudio/data/data16287/thu_news"
class ThuNews(TextClassificationDataset):
    def __init__(self, tokenizer, mode='train', max_seq_len=128):
        if mode == 'train':
            data_file = 'train.txt'
        elif mode == 'test':
            data_file = 'test.txt'
        else:
            data_file = 'valid.txt'
        super(ThuNews, self).__init__(
            base_path=DATA_DIR,
            data_file=data_file,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            is_file_with_header=True,
            label_list=['体育', '科技', '社会', '娱乐', '股票', '房产', '教育', '时政', '财经', '星座', '游戏', '家居', '彩票', '时尚'])

    # 解析文本文件里的样本
    def _read_file(self, input_file, is_file_with_header: bool = False):
        if not os.path.exists(input_file):
            raise RuntimeError("The file {} is not found.".format(input_file))
        else:
            with io.open(input_file, "r", encoding="UTF-8") as f:
                reader = csv.reader(f, delimiter="\t", quotechar=None)
                examples = []
                seq_id = 0
                header = next(reader) if is_file_with_header else None
                for line in reader:
                    example = InputExample(guid=seq_id, text_a=line[0], label=line[1])
                    seq_id += 1
                    examples.append(example)
                return examples

train_dataset = ThuNews(model.get_tokenizer(), mode='train', max_seq_len=128)
dev_dataset = ThuNews(model.get_tokenizer(), mode='dev', max_seq_len=128)
test_dataset = ThuNews(model.get_tokenizer(), mode='test', max_seq_len=128)
#打印测试集train.txt的前三个样本内容
for e in train_dataset.examples[:3]:
    print(e)

[2021-11-16 14:15:19,772] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2021-11-16 14:16:40,215] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt
[2021-11-16 14:16:51,212] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


text=主旋律在突破中赢得掌声江青正面形象亮相荧屏　　新中国成立60周年之际，银幕、荧屏、舞台上，各种献礼作品精彩纷呈。这些以历史为依据的艺术作品，或者人性化地展现伟人的性格，或者客观地诠释有争议性的人物，或者不回避历史，较以往艺术创作都有很大突破，而观众也看得非常过瘾。　　文/记者周娴、莫斯其格、张素芹　　很客观《解放》还原“争议”人物　　电视剧《解放》追求全景式展现历史，真实还原人物的宗旨，对很多颇受争议的历史人物做了客观的展现，江青首次以青春美好的正面形象出现在荧屏上。同时，该剧用大量篇幅表现了国民党中一些能征善战的将领，客观描绘了他们的性格特征，展现了他们作为军人的英勇与悲壮。相对于此类作品以往的“脸谱化”表现而言，这是一个突破。　　首次看到江青的正面形象　　在《解放》中，江青30多年来首次以正面形象出现在荧屏上，刚刚踏入解放战争洪流的江青，活泼开朗的性格被重点放大。对丈夫的崇拜和深情，照顾病中周恩来的细心周到，这些情节，都将少女江青的情怀刻画得淋漓尽致。剧情进入到中段，她与毛泽东及女儿的温馨生活场面则成为惨烈战争的调和剂。　　网友意见：《解放》里江青有大量的戏，她看上很纯很天真，这可是第一次啊。只有这样才能显得作品真实可信，应该客观公正地评价每一个人。　　主创回应：导演唐国强说，《解放》是站在唯物主义的历史观上，在特定的革命阶段，展现人物固有性格特征，还原人物的本来面貌。　　林彪与将士高唱军歌　　林彪是一代军事家，十大元帅中最年轻的元帅。10年前《大决战》中，首度出现林彪的正面形象，运筹帷幄，决胜千里。《解放》中，又让观众看到了林彪与将士们一起高唱“向前向前向前，我们的队伍向太阳”的景象，看到了一个更加不一样的林彪。　　网友意见：解放战争林彪功不可没，电视剧应该好好表现他。“千秋功罪，自有后人评说”。《解放》对林彪有了比较客观的评价，演员的诠释也相当出色，完全表现出一个具有卓越军事才能、个性孤僻又服从大局的军事家的感觉。　　主创回应：林彪的饰演者由立平说，为了接近林彪的身材，本来体重就不足130斤的我还是咬牙减去了10斤。保持在这个体重我心里才踏实，不光形体像了，而且通过减肥，似乎更能感受到他当时的内心压力。我对军事家有着独特的情感，我的这份情感不单单是向一个人，而是向整个老一辈军事家——致敬！　　国民党将领的铁骨柔情　　剧情进入中段后，《解放》将大量笔

In [5]:
## 迁移学习训练，本次为第二次训练，每次训练有3epochs
lert=5e-5
optimizer = paddle.optimizer.Adam(learning_rate=lert, parameters=model.parameters())
trainer = hub.Trainer(model, optimizer, checkpoint_dir='./ckpt', use_gpu=True)
trainer.train(train_dataset, epochs=3, batch_size=32, eval_dataset=dev_dataset, save_interval=1)   # 配置训练参数，启动训练，并指定验证集


[2021-11-16 14:17:03,071] [    INFO] - PaddleHub model checkpoint loaded. current_epoch=3 [acc=0.9021]
[2021-11-16 14:17:20,409] [   TRAIN] - Epoch=4/3, Step=10/282 loss=0.0572 acc=0.9812 lr=0.000050 step/sec=0.89 | ETA 00:15:52
[2021-11-16 14:17:27,611] [   TRAIN] - Epoch=4/3, Step=20/282 loss=0.0682 acc=0.9812 lr=0.000050 step/sec=1.39 | ETA 00:13:00
[2021-11-16 14:17:34,829] [   TRAIN] - Epoch=4/3, Step=30/282 loss=0.0546 acc=0.9875 lr=0.000050 step/sec=1.39 | ETA 00:12:04
[2021-11-16 14:17:42,041] [   TRAIN] - Epoch=4/3, Step=40/282 loss=0.0217 acc=0.9938 lr=0.000050 step/sec=1.39 | ETA 00:11:35
[2021-11-16 14:17:49,255] [   TRAIN] - Epoch=4/3, Step=50/282 loss=0.0494 acc=0.9906 lr=0.000050 step/sec=1.39 | ETA 00:11:18
[2021-11-16 14:17:56,454] [   TRAIN] - Epoch=4/3, Step=60/282 loss=0.0917 acc=0.9812 lr=0.000050 step/sec=1.39 | ETA 00:11:07
[2021-11-16 14:18:03,684] [   TRAIN] - Epoch=4/3, Step=70/282 loss=0.0696 acc=0.9844 lr=0.000050 step/sec=1.38 | ETA 00:10:59
[2021-11-16 14:

In [6]:
## 使用test数据集进行预测，3epochs最优模型结果精度为89.93%，6epochs最优模型结果精度为92.07%
result = trainer.evaluate(test_dataset, batch_size=32) 

[2021-11-16 14:28:40,111] [    EVAL] - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation on validation dataset: | - Evaluation on validation dataset: / - Evaluation on validation dataset: - - Evaluation on validation dataset: \ - Evaluation 

In [7]:
# 指定数据进行预测，使用保存的最佳模型
data = [
    # 房产
    ["昌平京基鹭府10月29日推别墅1200万套起享97折　　新浪房产讯(编辑郭彪)京基鹭府(论坛相册户型样板间点评地图搜索)售楼处位于昌平区京承高速北七家出口向西南公里路南。项目预计10月29日开盘，总价1200万元/套起，2012年年底入住。待售户型为联排户型面积为410-522平方米，独栋户型面积为938平方米，双拼户型面积为522平方米。　　京基鹭府项目位于昌平定泗路与东北路交界处。项目周边配套齐全，幼儿园：伊顿双语幼儿园、温莎双语幼儿园；中学：北师大亚太实验学校、潞河中学(北京市重点)；大学：王府语言学校、北京邮电大学、现代音乐学院；医院：王府中西医结合医院(三级甲等)、潞河医院、解放军263医院、安贞医院昌平分院；购物：龙德广场、中联万家商厦、世纪华联超市、瑰宝购物中心、家乐福超市；酒店：拉斐特城堡、鲍鱼岛；休闲娱乐设施：九华山庄、温都温泉度假村、小汤山疗养院、龙脉温泉度假村、小汤山文化广场、皇港高尔夫、高地高尔夫、北鸿高尔夫球场；银行：工商银行、建设银行、中国银行、北京农村商业银行；邮局：中国邮政储蓄；其它：北七家建材城、百安居建材超市、北七家镇武装部、北京宏翔鸿企业孵化基地等，享受便捷生活。"],
    # 游戏
    ["尽管官方到今天也没有公布《使命召唤：现代战争2》的游戏详情，但《使命召唤：现代战争2》首部包含游戏画面的影片终于现身。虽然影片仅有短短不到20秒，但影片最后承诺大家将于美国时间5月24日NBA职业篮球东区决赛时将会揭露更多的游戏内容。　　这部只有18秒的广告片闪现了9个镜头，能够辨识的场景有直升机飞向海岛军事工事，有飞机场争夺战，有潜艇和水下工兵，有冰上乘具，以及其他的一些镜头。整体来看《现代战争2》很大可能仍旧与俄罗斯有关。　　片尾有一则预告：“May24th，EasternConferenceFinals”，这是什么？这是说当前美国NBA联赛东部总决赛的日期。原来这部视频是NBA季后赛奥兰多魔术对波士顿凯尔特人队时，TNT电视台播放的广告。"],
    # 体育
    ["罗马锋王竟公然挑战两大旗帜拉涅利的球队到底错在哪　　记者张恺报道主场一球小胜副班长巴里无可吹捧，罗马占优也纯属正常，倒是托蒂罚失点球和前两号门将先后受伤(多尼以三号身份出场)更让人揪心。阵容规模扩大，反而表现不如上赛季，缺乏一流强队的色彩，这是所有球迷对罗马的印象。　　拉涅利说：“去年我们带着嫉妒之心看国米，今年我们也有了和国米同等的超级阵容，许多教练都想有罗马的球员。阵容广了，寻找队内平衡就难了，某些时段球员的互相排斥和跟从前相比的落差都正常。有好的一面，也有不好的一面，所幸，我们一直在说一支伟大的罗马，必胜的信念和够级别的阵容，我们有了。”拉涅利的总结由近一阶段困扰罗马的队内摩擦、个别球员闹意见要走人而发，本赛季技术层面强化的罗马一直没有上赛季反扑的面貌，内部变化值得球迷关注。"],
    # 教育
    ["新总督致力提高加拿大公立教育质量　　滑铁卢大学校长约翰斯顿先生于10月1日担任加拿大总督职务。约翰斯顿先生还曾任麦吉尔大学长，并曾在多伦多大学、女王大学和西安大略大学担任教学职位。　　约翰斯顿先生在就职演说中表示，要将加拿大建设成为一个“聪明与关爱的国度”。为实现这一目标，他提出三个支柱：支持并关爱家庭、儿童；鼓励学习与创造；提倡慈善和志愿者精神。他尤其强调要关爱并尊重教师，并通过公立教育使每个人的才智得到充分发展。"]
]

label_list=['体育', '科技', '社会', '娱乐', '股票', '房产', '教育', '时政', '财经', '星座', '游戏', '家居', '彩票', '时尚']
label_map = { 
    idx: label_text for idx, label_text in enumerate(label_list)
}

model = hub.Module(
    name='ernie',
    task='seq-cls',
    load_checkpoint='./ckpt/best_model/model.pdparams',
    label_map=label_map)
results = model.predict(data, max_seq_len=128, batch_size=1, use_gpu=True)
for idx, text in enumerate(data):
    print('Data: {} \nLable: {}\n'.format(text[0], results[idx]))

[2021-11-16 14:28:40,122] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-11-16 14:28:44,835] [    INFO] - Loaded parameters from /home/aistudio/data/data16287/ckpt/best_model/model.pdparams
[2021-11-16 14:28:44,842] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


Data: 昌平京基鹭府10月29日推别墅1200万套起享97折　　新浪房产讯(编辑郭彪)京基鹭府(论坛相册户型样板间点评地图搜索)售楼处位于昌平区京承高速北七家出口向西南公里路南。项目预计10月29日开盘，总价1200万元/套起，2012年年底入住。待售户型为联排户型面积为410-522平方米，独栋户型面积为938平方米，双拼户型面积为522平方米。　　京基鹭府项目位于昌平定泗路与东北路交界处。项目周边配套齐全，幼儿园：伊顿双语幼儿园、温莎双语幼儿园；中学：北师大亚太实验学校、潞河中学(北京市重点)；大学：王府语言学校、北京邮电大学、现代音乐学院；医院：王府中西医结合医院(三级甲等)、潞河医院、解放军263医院、安贞医院昌平分院；购物：龙德广场、中联万家商厦、世纪华联超市、瑰宝购物中心、家乐福超市；酒店：拉斐特城堡、鲍鱼岛；休闲娱乐设施：九华山庄、温都温泉度假村、小汤山疗养院、龙脉温泉度假村、小汤山文化广场、皇港高尔夫、高地高尔夫、北鸿高尔夫球场；银行：工商银行、建设银行、中国银行、北京农村商业银行；邮局：中国邮政储蓄；其它：北七家建材城、百安居建材超市、北七家镇武装部、北京宏翔鸿企业孵化基地等，享受便捷生活。 
Lable: 房产

Data: 尽管官方到今天也没有公布《使命召唤：现代战争2》的游戏详情，但《使命召唤：现代战争2》首部包含游戏画面的影片终于现身。虽然影片仅有短短不到20秒，但影片最后承诺大家将于美国时间5月24日NBA职业篮球东区决赛时将会揭露更多的游戏内容。　　这部只有18秒的广告片闪现了9个镜头，能够辨识的场景有直升机飞向海岛军事工事，有飞机场争夺战，有潜艇和水下工兵，有冰上乘具，以及其他的一些镜头。整体来看《现代战争2》很大可能仍旧与俄罗斯有关。　　片尾有一则预告：“May24th，EasternConferenceFinals”，这是什么？这是说当前美国NBA联赛东部总决赛的日期。原来这部视频是NBA季后赛奥兰多魔术对波士顿凯尔特人队时，TNT电视台播放的广告。 
Lable: 游戏

Data: 罗马锋王竟公然挑战两大旗帜拉涅利的球队到底错在哪　　记者张恺报道主场一球小胜副班长巴里无可吹捧，罗马占优也纯属正常，倒是托蒂罚失点球和前两号门将先后受伤(多尼以三号身份出场)更让人揪心。阵容规模扩大，反而表现不如上赛季，缺乏一流强队的色彩，这是